In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
import itertools

In [ ]:
tools_dir_path = os.path.abspath('')
root_dir_path = os.path.dirname(tools_dir_path)
results_dir_path = os.path.join(root_dir_path, 'results')

In [ ]:
cur_data_dir_path = os.path.join(results_dir_path, 'data-3990x')

In [ ]:
filename_list = [f for f in os.listdir(cur_data_dir_path) if os.path.isfile(os.path.join(cur_data_dir_path, f))]
data_filename_pattern = re.compile(r'(.+)__(.+)__(.+)__(.+)\.csv')
data_filename_list = []
data_filepath_list = []
map_name_list = []
hash_name_list = []
key_name_list = []
value_name_list = []
# k_v_name_list = []
k_v_dict = {}
for filename in filename_list:
    match_ret = data_filename_pattern.findall(filename)
    if match_ret:
        data_file_path = os.path.join(cur_data_dir_path, filename)
        data_filepath_list.append(data_file_path)
        data_filename_list.append(filename)
        temp_map_name = match_ret[0][0]
        temp_hash_name = match_ret[0][1]
        map_name_list.append(temp_map_name)
        hash_name_list.append(temp_hash_name)
        key_name_list.append(match_ret[0][2])
        value_name_list.append(match_ret[0][3])
        temp_k_v_name = '<' + match_ret[0][2] + ',' + match_ret[0][3] + '>'
        # k_v_name_list.append(temp_k_v_name)
        if not temp_k_v_name in k_v_dict:
            k_v_dict[temp_k_v_name] = {'data_filepath_list':[], 'hash_name_list':[], 'map_name_list':[]}
        k_v_dict[temp_k_v_name]['data_filepath_list'].append(data_file_path)
        k_v_dict[temp_k_v_name]['hash_name_list'].append(temp_hash_name)
        k_v_dict[temp_k_v_name]['map_name_list'].append(temp_map_name)
print("total %d data files" % (len(data_filename_list)))
map_name_set = set(map_name_list)
hash_name_set = set(hash_name_list)
# k_v_name_set = set(k_v_name_list)
print(data_filename_list)
# print(map_name_list)
# print(hash_name_list)
# print(key_name_list)
# print(value_name_list)
print(map_name_set)
print(hash_name_set)
print(k_v_dict.keys())

In [ ]:
def plot_lines(title, x_arr_list, y_arr_list, label_list, y_axis_label):
    fig, ax = plt.subplots(figsize=(8, 6), dpi=300)
    fig.suptitle(title, fontsize='small')
    assert(len(x_arr_list) == len(y_arr_list))
    arr_num = len(x_arr_list)
    marker = itertools.cycle(('v','o','^','<','>','s','p','*','+','x','D')) 
    for i in range(arr_num):
        ax.plot(x_arr_list[i], y_arr_list[i], label=label_list[i], marker=next(marker), markersize=3.0, linewidth=1.0)
    ax.set_xlabel('element num')
    ax.set_ylabel(y_axis_label)
    ax.set_yscale('log')
    ax.set_xscale('log')
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])
    fig.legend(loc='lower center', fontsize='xx-small')

In [ ]:
for k_v_name, data_dict in k_v_dict.items():
    # print(k_v_name)
    data_file_num = len(data_dict['data_filepath_list'])
    map_hash_name_list = []
    element_num_arr_list = []
    construct_time_with_reserve_arr_list = []
    construct_time_without_reserve_arr_list = []
    hit_without_rehash_lookup_arr_list = []
    miss_without_rehash_lookup_arr_list = []
    hit_with_rehash_lookup_arr_list = []
    miss_with_rehash_lookup_arr_list = []
    iterate_arr_list = []
    with_final_rehash_construct_arr_list= []
    for t in range(data_file_num):
        temp_map_hash_name = data_dict['map_name_list'][t] + "," + data_dict['hash_name_list'][t]
        map_hash_name_list.append(temp_map_hash_name)
        temp_data_filepath = data_dict['data_filepath_list'][t]
        # print(temp_data_filepath)
        temp_data_df = pd.read_csv(temp_data_filepath, sep=',')
        temp_data_df = temp_data_df.rename(columns=lambda x: x.strip())
        # print(temp_data_df.head())
        element_num_arr_list.append(temp_data_df['element_num'])
        construct_time_with_reserve_arr_list.append(temp_data_df['avg_construct_time_with_reserve_ns'])
        construct_time_without_reserve_arr_list.append(temp_data_df['avg_construct_time_without_reserve_ns'])
        hit_without_rehash_lookup_arr_list.append(temp_data_df['avg_hit_without_rehash_lookup_ns'])
        miss_without_rehash_lookup_arr_list.append(temp_data_df['avg_miss_without_rehash_lookup_ns'])
        hit_with_rehash_lookup_arr_list.append(temp_data_df['avg_hit_with_rehash_lookup_ns'])
        miss_with_rehash_lookup_arr_list.append(temp_data_df['avg_miss_with_rehash_lookup_ns'])
        iterate_arr_list.append(temp_data_df['avg_iterate_ns'])
        with_final_rehash_construct_arr_list.append(temp_data_df['avg_with_final_rehash_construct_ns'])
    plot_lines(k_v_name + ', avg_insert_time_with_reserve', element_num_arr_list, construct_time_with_reserve_arr_list, map_hash_name_list, 'ns')
    plot_lines(k_v_name + ', avg_insert_time_without_reserve', element_num_arr_list, construct_time_without_reserve_arr_list, map_hash_name_list, 'ns')
    plot_lines(k_v_name + ', avg_hit_find_without_rehash', element_num_arr_list, hit_without_rehash_lookup_arr_list, map_hash_name_list, 'ns')
    plot_lines(k_v_name + ', avg_miss_find_without_rehash', element_num_arr_list, miss_without_rehash_lookup_arr_list, map_hash_name_list, 'ns')
    plot_lines(k_v_name + ', avg_hit_find_with_rehash', element_num_arr_list, hit_with_rehash_lookup_arr_list, map_hash_name_list, 'ns')
    plot_lines(k_v_name + ', avg_miss_find_with_rehash', element_num_arr_list, miss_with_rehash_lookup_arr_list, map_hash_name_list, 'ns')
    plot_lines(k_v_name + ', avg_iterate', element_num_arr_list, iterate_arr_list, map_hash_name_list, 'ns')
    plot_lines(k_v_name + ', avg_construct_time_with_final_rehash', element_num_arr_list, with_final_rehash_construct_arr_list, map_hash_name_list, 'ns')


In [ ]:
data_file_num = len(data_filename_list)
for k in range(data_file_num):
    temp_data_file_path = data_filepath_list[k]
    temp_data_df = pd.read_csv(temp_data_file_path, sep=',')
    break
temp_data_df